In [1]:
import pandas as pd 
import requests
from bs4 import BeautifulSoup

# Getting real estate listings


## Zumber

In [2]:
def Zumber(content): 
    soup = BeautifulSoup(content, "html.parser")
    rent = [x.text for x in soup.find_all("div", class_="css-uzc059")]
    bath = [x.text for x in soup.find_all("div", class_="css-1iq6kk8")]


    dict = {'Monthly': rent, 'Number of baths': bath}
    df= pd.DataFrame(dict)
    df['Monthly'] = df['Monthly'].str.replace('\D+', '')
    return df

### Studios

In [3]:
URL = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/studios"
requestURL = requests.get(URL)
content = requestURL.content

In [4]:
Zumber(content)

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_34684/506841304.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = df['Monthly'].str.replace('\D+', '')


,Monthly,Number of baths
0,1900,Studio • 1 Bath
1,1625,Studio • 1 Bath
2,2000,Studio • 1 Bath
3,2369,Studio • 1 Bath
4,2268,Studio • 1 Bath
5,2570,Studio • 1 Bath
6,2650,Studio • 1 Bath
7,2480,Studio • 1 Bath
8,2495,Studio • 1 Bath
9,3500,Studio • 1 Bath


### 1 bed

In [5]:
URL2 = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/1-beds"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

In [6]:
Zumber(content2)

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_34684/506841304.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = df['Monthly'].str.replace('\D+', '')


,Monthly,Number of baths
0,3150,1 Bed • 1 Bath
1,2450,1 Bed • 1 Bath
2,3875,1 Bed • 1 Bath
3,2680,1 Bed • 1 Bath
4,2400,1 Bed • 1 Bath
5,2200,1 Bed • 1 - 2 Baths
6,2750,1 Bed • 1 Bath
7,2614,1 Bed • 1 Bath
8,3000,1 Bed • 1 Bath
9,2629,1 Bed • 1 Bath


## 2 Beds

In [7]:
URL3 = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/2-beds"
requestURL3 = requests.get(URL3)
content3 = requestURL3.content


In [8]:
Zumber(content3)

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_34684/506841304.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = df['Monthly'].str.replace('\D+', '')


,Monthly,Number of baths
0,4300,2 Beds • 1 - 2 Baths
1,4100,2 Beds • 1 - 2 Baths
2,4500,2 Beds • 2 Baths
3,3500,2 Beds • 1 - 2 Baths
4,3400,2 Beds • 1 Bath
5,3300,2 Beds • 1 - 2 Baths
6,7800,2 Beds • 2 Baths
7,265027,2 Beds • 1 Bath
8,4900,2 Beds • 2 Baths
9,3895,2 Beds • 2 Baths


### 3 beds

In [9]:
URL4 = "https://www.zumper.com/apartments-for-rent/vancouver-bc/west-end/3-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content


In [10]:
Zumber(content4)

C:\Users\Jimmy\AppData\Local\Temp/ipykernel_34684/506841304.py:9: FutureWarning: The default value of regex will change from True to False in a future version.
  df['Monthly'] = df['Monthly'].str.replace('\D+', '')


,Monthly,Number of baths
0,6550,3 Beds • 2 Baths
1,4500,3 Beds • 2 Baths
2,129,3 Beds • 3 Baths
3,5195,3 Beds • 2 Baths
4,6200,3 Beds • 2 Baths
5,5000,3 Beds • 2 Baths
6,4500,3 Beds • 2 Baths
7,4500,3 Beds • 2 Baths
8,4900,3 Beds • 1 Bath
9,430012,3 Beds • 2 Baths


## Kijiji

In [11]:
def kijiji(content): 
    soup = BeautifulSoup(content, "lxml")
    realEstate = soup.findAll("div", class_="container-results large-images")[1]
    realEstate2 = [x.find('div', class_="info") for x in realEstate.findAll("div", attrs={"data-listing-id":True})]

    realEstateLinks = [x.find('div', class_=["title"]) for x in realEstate2]
    realEstateLinks1 = [x.find('a', attrs={"href":True}) for x in realEstate2]
    realEstateLinks = ["https://www.kijiji.ca"+ x['href'] for x in realEstateLinks1]

    dfKijiji = pd.DataFrame(columns=['Monthly', 'Number of baths'])

    for i in range(len(realEstateLinks)):
        requestsIterate = requests.get(realEstateLinks[i])
        soup3 = BeautifulSoup(requestsIterate.text, "html.parser")


        for x in soup3.find_all('div', class_=['realEstateTitle-1440881021']):
            monthlyNum = x.find('span')['content']
            bathroomNum = x.findAll('span', class_="noLabelValue-3861810455")[-1].text

            dfKijiji = dfKijiji.append({'Monthly': monthlyNum, 'Number of baths': bathroomNum}, ignore_index=True)
    return dfKijiji
    



### Studios

In [12]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/bachelor+studio/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
content2 = requestURL2.content

In [13]:
kijiji(content2)

,Monthly,Number of baths
0,2268.,Bathrooms: 1
1,2650.,Bathrooms: 1
2,1640.,Bathrooms: 1
3,2369.,Bathrooms: 1
4,2195.,Bathrooms: 1
5,2375.,Bathrooms: 1
6,2325.,Bathrooms: 1
7,2900.,Bathrooms: 1
8,2570.,Bathrooms: 1
9,2510.,Bathrooms: 1


### 1 bedroom

In [14]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/1+bedroom/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
contentOneBedRoom = requestURL2.content

In [15]:
kijiji(contentOneBedRoom)

,Monthly,Number of baths
0,2614.,Bathrooms: 1
1,2743.,Bathrooms: 1
2,2599.,Bathrooms: 1
3,2600.,Bathrooms: 1
4,2750.,Bathrooms: 1
5,2300.,Bathrooms: 1
6,2532.,Bathrooms: 1
7,2629.,Bathrooms: 1
8,2500.,Bathrooms: 1
9,2550.,Bathrooms: 1


### 2 bed

In [18]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/2+bedrooms-apartment/c37l1700292a27949001a29276001"
requestURL2 = requests.get(URL2)
contentTwoBedRoom = requestURL2.content

In [19]:
kijiji(contentTwoBedRoom)

,Monthly,Number of baths
0,5975.,Bathrooms: 2
1,6680.,Bathrooms: 3
2,2399.,Bathrooms: 1
3,3650.,Bathrooms: 1
4,3485.,Bathrooms: 2
5,3799.,Bathrooms: 1
6,3817.,Bathrooms: 1
7,3262.,Bathrooms: 1
8,3428.,Bathrooms: 1.5
9,3599.,Bathrooms: 1


### 3 bed

In [20]:
URL2 = "https://www.kijiji.ca/b-apartments-condos/vancouver-downtown-west-end/3+bedroom/c37l1700292a27949001"
requestURL2 = requests.get(URL2)
contentThreeBedRoom = requestURL2.content

In [21]:
kijiji(contentThreeBedRoom)

,Monthly,Number of baths
0,2980.,Bathrooms: 1
1,2614.,Bathrooms: 1
2,2268.,Bathrooms: 1
3,2743.,Bathrooms: 1
4,5800.,Bathrooms: 2
5,5975.,Bathrooms: 2
6,4500.,Bathrooms: 2
7,6550.,Bathrooms: 2
8,3875.,Bathrooms: 1
9,6680.,Bathrooms: 3


## Pad Mapper

In [22]:
def padMapper(content): 
    soupPad = BeautifulSoup(content, "html.parser").find('body')

    realEstate = soupPad.findAll("div", class_="ListItemMobile_text__3KFu_")
    realEstate2 = [x.text.strip() for x in realEstate]
    return realEstate2

### Studio 

In [23]:
URL3 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/studios"
requestURL3 = requests.get(URL3)
content3 = requestURL3.content

In [24]:
padMapper(content3)

['$1,900+',
 '$2,369',
 '$2,495',
 '$2,700+',
 '$2,200',
 '$2,268',
 '$2,500',
 '$2,095',
 '$1,625',
 '$2,000',
 '$2,570',
 '$2,650',
 '$2,480',
 '$2,650',
 '$3,500']

### 1 bathrooms

In [25]:
URL4 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/1-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content

padMapper(content4)

['$2,680+',
 '$3,150+',
 '$3,875',
 '$2,450+',
 '$2,355+',
 '$2,200+',
 '$2,325+',
 '$2,629',
 '$2,889',
 '$2,400',
 '$2,706',
 '$2,743',
 '$2,600',
 '$2,850',
 '$2,550',
 '$2,375+',
 '$2,750+',
 '$2,614',
 '$2,695',
 '$2,532',
 '$2,361',
 '$2,550',
 '$2,200']

### 2 bathrooms

In [26]:
URL4 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/2-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content

padMapper(content4)

['$4,500+',
 '$4,100+',
 '$4,300+',
 '$3,495',
 '$4,900',
 '$3,599',
 '$3,799',
 '$3,895',
 '$4,850',
 '$3,428',
 '$3,500+',
 '$3,400',
 '$3,300+',
 '$3,262',
 '$3,650+',
 '$3,450',
 '$2,650',
 '$3,050',
 '$3,900',
 '$7,800',
 '$7,500',
 '$11K',
 '$3,250']

In [27]:
URL4 = "https://www.padmapper.com/apartments/vancouver-bc/west-end/3-beds"
requestURL4 = requests.get(URL4)
content4 = requestURL4.content

padMapper(content4)

['$6,550',
 '$4,500',
 '$5,195',
 '$6,200',
 '$4,500+',
 '$5,000',
 '$4,500',
 '$12.9K',
 '$4,900',
 '$4,300']

## Rent cafe

In [28]:
def rentCafe(content): 
    soupCafe = BeautifulSoup(content)
    realEstate = soupCafe.findAll("li", class_="data-rent")
    realEstate2 = [x.text.strip() for x in realEstate]
    return realEstate2
    

In [29]:
URLCafe1 = "https://www.rentcafe.com/studio-apartments-for-rent/ca/bc/vancouver/west-end-vancouver/"
requestURLCafe1 = requests.get(URLCafe1)
contentCafe1 = requestURLCafe1.content

rentCafe(contentCafe1)

['$2,375', '$2,025', '$2,240+']

In [30]:
URLCafe1 = "https://www.rentcafe.com/1-bedroom-apartments-for-rent/ca/bc/vancouver/west-end-vancouver/"
requestURLCafe1 = requests.get(URLCafe1)
contentCafe1 = requestURLCafe1.content

rentCafe(contentCafe1)

['$2,350+',
 '$2,175+',
 '$2,950',
 '$2,800',
 '$2,700',
 '$2,500',
 '$2,275',
 '$2,270+',
 '$2,100']

In [31]:
URLCafe1 = "https://www.rentcafe.com/2-bedroom-apartments-for-rent/ca/bc/vancouver/west-end-vancouver/"
requestURLCafe1 = requests.get(URLCafe1)
contentCafe1 = requestURLCafe1.content

rentCafe(contentCafe1)

['$3,650',
 '$3,050+',
 '$4,500',
 '$3,800',
 '$2,650',
 '$3,000',
 '$3,250+',
 '$2,800',
 '$2,950+',
 '$2,100']

In [32]:
URLCafe1 = "https://www.rentcafe.com/3-bedroom-apartments-for-rent/ca/bc/vancouver/west-end-vancouver/"
requestURLCafe1 = requests.get(URLCafe1)
contentCafe1 = requestURLCafe1.content

rentCafe(contentCafe1)

['$6,500', '$4,200', '$12,000', '$2,850', '$4,050', '$3,700']